In [4]:
import json
import csv

def extract_tweet_info_from_json(json_file, csv_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
        with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['tweet_id', 'username', 'created_at']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for tweet_data in data:
                tweet_id = tweet_data['tweet']['id_str']
                user_mentions = tweet_data['tweet'].get('entities', {}).get('user_mentions', [])
                if user_mentions:
                    username = user_mentions[0]['screen_name']
                else:
                    username = "N/A"
                created_at = tweet_data['tweet']['created_at']
                writer.writerow({'tweet_id': tweet_id, 'username': username, 'created_at': created_at})

if __name__ == "__main__":
    # Replace 'tweets.json' with the path to your JSON archive file
    json_file = 'data.json'
    # Replace 'tweet_info.csv' with the desired name for your CSV file
    csv_file = 'tweet_IDs.csv'
    extract_tweet_info_from_json(json_file, csv_file)
    print("Data has been written to", csv_file)


Data has been written to tweet_IDs.csv


In [5]:
import csv
import requests
from bs4 import BeautifulSoup
import urllib3
import time

# Disable SSL certificate verification warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def scrape_nitter_from_csv(csv_file):
    nitter_instances = [
        "https://scraped-knees.com"
        # Add more Nitter instances from the provided list as needed
    ]

    for instance in nitter_instances:
        print(f"Trying Nitter instance: {instance}")
        success = scrape_nitter_tweets(instance, csv_file)
        if success:
            break
        else:
            print(f"Failed with instance: {instance}. Retrying...")

def scrape_nitter_tweets(nitter_instance, csv_file):
    with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tweet_id = row['tweet_id']
            success = scrape_nitter_tweet(nitter_instance, tweet_id)
            if not success:
                return False
            # Delay before scraping the next tweet (to avoid overloading the instance)
            time.sleep(10)  # Add a delay of 1 second between requests
    return True

def scrape_nitter_tweet(nitter_instance, tweet_id):
    base_url = f"{nitter_instance}/twitter/status/"
    url = base_url + str(tweet_id)
    try:
        response = requests.get(url, verify=False, timeout=10)  # Disable SSL certificate verification
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Example: Scraping the tweet text
            tweet_text = soup.find('div', class_='tweet-text').text.strip()
            print("Tweet ID:", tweet_id)
            print("Tweet Text:", tweet_text)
            print("-" * 50)
            return True
        else:
            print(f"Failed to fetch tweet ID {tweet_id} from {nitter_instance}. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"An error occurred while fetching tweet ID {tweet_id} from {nitter_instance}: {e}")
        return False

if __name__ == "__main__":
    # Replace 'tweet_info.csv' with the path to your CSV file containing tweet IDs
    csv_file = 'tweet_IDs.csv'
    scrape_nitter_from_csv(csv_file)

Trying Nitter instance: https://scraped-knees.com
An error occurred while fetching tweet ID 1758594167708832194 from https://scraped-knees.com: 'NoneType' object has no attribute 'text'
Failed with instance: https://scraped-knees.com. Retrying...
